# Import Libraries

In [1]:
import glob
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, f1_score, precision_score, recall_score

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cuda


In [3]:
df_full = pd.read_parquet('../BIS_data/Final_BIS_Data.parquet')
df_full = df_full.reset_index(drop=True)
df_full.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2138275 entries, 0 to 2138274
Data columns (total 6 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   pca_0                   float64
 1   pca_1                   float64
 2   pca_2                   float64
 3   pca_3                   float64
 4   pca_4                   float64
 5   laundering_schema_type  int64  
dtypes: float64(5), int64(1)
memory usage: 97.9 MB


# 2. Split Data

In [4]:
# ---------------------------
# 2. Split Data into Train and Test Sets BEFORE Undersampling
# ---------------------------
X = df_full.drop('laundering_schema_type', axis=1)
y = df_full['laundering_schema_type']

# Use stratify=y to preserve the class distribution in the test set.
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.1, random_state=42, stratify=y
)

# 3. Undersampling

In [5]:
from imblearn.under_sampling import RandomUnderSampler

# ---------------------------
# 3. Undersample Only the Training Set
# ---------------------------
# Combine training features and labels for processing.
train_df = X_train.copy()
train_df['laundering_schema_type'] = y_train

# Separate minority (fraud) and majority (non-fraud) classes in the training data.
fraud_train = train_df[train_df['laundering_schema_type'] == 1]

# Undersample the majority class to match the number of fraud samples.
n_samples = len(fraud_train)


rus = RandomUnderSampler(sampling_strategy={0:n_samples}, random_state=42)
X_under, y_under = rus.fit_resample(X_train.values, y_train.values)

train_df_undersampled = pd.DataFrame(X_under, columns=[f'pca_{i}' for i in range(X_under.shape[1])])
train_df_undersampled['laundering_schema_type'] = y_under


C:\Users\User\anaconda3\envs\BIS_env\lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
C:\Users\User\anaconda3\envs\BIS_env\lib\site-packages\sklearn\base.py:484: FutureWarning: `BaseEstimator._check_n_features` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation._check_n_features` instead.
  warnings.warn(
C:\Users\User\anaconda3\envs\BIS_env\lib\site-packages\sklearn\base.py:493: FutureWarning: `BaseEstimator._check_feature_names` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation._check_feature_names` instead.
  warnings.warn(


In [6]:
# Separate features and labels for the undersampled training data.
X_train = train_df_undersampled.drop('laundering_schema_type', axis=1).values
y_train = train_df_undersampled['laundering_schema_type'].values
X_train = X_train.astype(np.float32)

# For the test set, keep the original (imbalanced) distribution.
X_test = X_test.values
y_test = y_test.values
X_test = X_test.astype(np.float32)

# Convert the datasets to PyTorch tensors.
X_train = torch.tensor(X_train, dtype=torch.float).to(device)
y_train = torch.tensor(y_train, dtype=torch.long).to(device)
X_test  = torch.tensor(X_test, dtype=torch.float).to(device)
y_test  = torch.tensor(y_test, dtype=torch.long).to(device)

# 4. Define Model

In [7]:
# ---------------------------
# 4. Define the MLP Model
# ---------------------------
class ResidualBlock(nn.Module):
    def __init__(self, hidden_dim):
        super(ResidualBlock, self).__init__()
        self.block = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim),
            nn.BatchNorm1d(hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.BatchNorm1d(hidden_dim),
        )
        self.relu = nn.ReLU()
        
    def forward(self, x):
        residual = x
        out = self.block(x)
        out += residual  # Skip connection
        out = self.relu(out)
        return out

class FraudResNet(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_blocks=3):
        super(FraudResNet, self).__init__()
        # Initial projection layer
        self.input_layer = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.BatchNorm1d(hidden_dim),
            nn.ReLU()
        )
        
        # Residual blocks
        self.res_blocks = nn.Sequential(
            *[ResidualBlock(hidden_dim) for _ in range(num_blocks)]
        )
        
        # Final classification layer
        self.output_layer = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, x):
        x = self.input_layer(x)
        x = self.res_blocks(x)
        x = self.output_layer(x)
        return x

In [8]:
input_dim = X_train.shape[1]
hidden_dim = 32
output_dim = 2  # Two neurons for two classes
num_blocks = 3  # Number of residual blocks

model = FraudResNet(input_dim, hidden_dim, output_dim, num_blocks).to(device)

In [9]:
# ---------------------------
# 5. Setup Loss, Optimizer, and Training Parameters
# ---------------------------
# Since the training data is now balanced, we can use the default weights.
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 200
best_loss = float('inf')
best_model_path = "./Model_Weight/ResNet_best_undersampling_BIS.pt"

In [10]:
# ---------------------------
# 6. Training Loop with Checkpointing (Saving Best Model by Training Loss)
# ---------------------------
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    
    outputs = model(X_train)
    loss = criterion(outputs, y_train)
    loss.backward()
    optimizer.step()
    
    # Save the model if training loss improved.
    if loss.item() < best_loss:
        best_loss = loss.item()
        torch.save(model.state_dict(), best_model_path)
    
    if (epoch + 1) % 5 == 0:
        print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}")

Epoch [5/200], Loss: 0.7774
Epoch [10/200], Loss: 0.5921
Epoch [15/200], Loss: 0.4929
Epoch [20/200], Loss: 0.4389
Epoch [25/200], Loss: 0.4066
Epoch [30/200], Loss: 0.3866
Epoch [35/200], Loss: 0.3718
Epoch [40/200], Loss: 0.3604
Epoch [45/200], Loss: 0.3516
Epoch [50/200], Loss: 0.3446
Epoch [55/200], Loss: 0.3390
Epoch [60/200], Loss: 0.3343
Epoch [65/200], Loss: 0.3303
Epoch [70/200], Loss: 0.3268
Epoch [75/200], Loss: 0.3238
Epoch [80/200], Loss: 0.3210
Epoch [85/200], Loss: 0.3185
Epoch [90/200], Loss: 0.3162
Epoch [95/200], Loss: 0.3141
Epoch [100/200], Loss: 0.3122
Epoch [105/200], Loss: 0.3104
Epoch [110/200], Loss: 0.3088
Epoch [115/200], Loss: 0.3072
Epoch [120/200], Loss: 0.3058
Epoch [125/200], Loss: 0.3045
Epoch [130/200], Loss: 0.3033
Epoch [135/200], Loss: 0.3021
Epoch [140/200], Loss: 0.3010
Epoch [145/200], Loss: 0.3000
Epoch [150/200], Loss: 0.2990
Epoch [155/200], Loss: 0.2981
Epoch [160/200], Loss: 0.2972
Epoch [165/200], Loss: 0.2964
Epoch [170/200], Loss: 0.2957


In [11]:
# ---------------------------
# 7. Evaluation on the Test Set (with the Original Imbalanced Distribution)
# ---------------------------
model.eval()
with torch.no_grad():
    test_outputs = model(X_test)
    _, predicted = torch.max(test_outputs, 1)
    
    y_pred = predicted.cpu().numpy()
    y_true = y_test.cpu().numpy()
    
    print("\nEvaluation on (Test Data):")
    print(classification_report(y_true, y_pred))
    print(confusion_matrix(y_true, y_pred))
    print("F1-score:", f1_score(y_true, y_pred))
    print("Precision-score:", precision_score(y_true, y_pred))
    print("Recall-score:", recall_score(y_true, y_pred))


Evaluation on (Test Data):
              precision    recall  f1-score   support

           0       0.99      0.86      0.92    200000
           1       0.31      0.87      0.45     13828

    accuracy                           0.86    213828
   macro avg       0.65      0.87      0.69    213828
weighted avg       0.95      0.86      0.89    213828

[[172688  27312]
 [  1768  12060]]
F1-score: 0.45338345864661656
Precision-score: 0.30630905211825665
Recall-score: 0.872143477003182


# 5. Evaluate on Unseen Data (Without Fine-tuning)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

In [ ]:
# Without Fine-tuning
# ---------------------------
# 8. Load the Best Model into a New Instance and Test on Unseen Data (e.g., SetB)
# ---------------------------
# Create a new model instance with the same architecture.
best_model_path = "./Model_Final/CNN_best_model_undersampling_final_BIS_V2.pt"

loaded_model = FraudResNet(5, 64, 2, 3).to(device)
loaded_model.load_state_dict(torch.load(best_model_path))
loaded_model.eval()


In [ ]:
npy_files_unseen_1 = glob.glob("./BIS_data/client_part7_1.npy")

# Load each npy file into a list
client_data_list_unseen_1 = [np.load(file, allow_pickle=True) for file in npy_files_unseen_1]

# Combine all client data into one numpy array
combined_data_unseen_1 = np.vstack(client_data_list_unseen_1)

col_names = [f'pca_{i}' for i in range(5)] + ['laundering_schema_type', 'laundering_schema_id']

# Convert the combined array into a DataFrame
new_df = pd.DataFrame(combined_data_unseen_1, columns=col_names)
new_df = new_df.drop('laundering_schema_id', axis=1)
new_df = new_df.sample(frac=1, random_state=42).reset_index(drop=True)
new_df['laundering_schema_type'] = new_df['laundering_schema_type'].notna().astype(int)
new_df = new_df.sample(frac=1)

In [ ]:

# Assume you have an unseen dataset (SetB) with the same attributes.
# new_df = pd.read_csv("Fraud_dataset/Creditcard/GlobalUnseen.csv")

if 'laundering_schema_type' in new_df.columns:
    X_new = new_df.drop('laundering_schema_type', axis=1).values
    y_new = new_df['laundering_schema_type'].values
else:
    X_new = new_df.values
    y_new = None

X_new = X_new.astype(np.float32)
X_new = torch.tensor(X_new, dtype=torch.float).to(device)
if y_new is not None:
    y_new = torch.tensor(y_new, dtype=torch.long).to(device)

with torch.no_grad():
    new_outputs = loaded_model(X_new)
    _, new_predicted = torch.max(new_outputs, 1)
    new_predictions = new_predicted.cpu().numpy()

if y_new is not None:
    # Convert y_new tensor to numpy array for metric calculations.
    y_new_np = y_new.cpu().numpy()
    print("\nEvaluation on Unseen Data (Set7_part1):")
    print(classification_report(y_new_np, new_predictions))
    print(confusion_matrix(y_new_np, new_predictions))
    print("F1-score:", f1_score(y_new_np, new_predictions))
    print("Precision-score:", precision_score(y_new_np, new_predictions))
    print("Recall-score:", recall_score(y_new_np, new_predictions))
else:
    print("\nPredictions on Unseen Data (Set7_part1):")
    print(new_predictions)

# 6. Evaluate on Unseen Data (With Fine-tuning)

In [ ]:
# new_df = pd.read_csv("Fraud_dataset/Creditcard/GlobalUnseen.csv")
# best_model_path = "./Model_Final/best_model_undersampling_final_all.pt"

# # Separate features and labels.
# X_new = new_df.drop('Class', axis=1).values
# y_new = new_df['Class'].values

# # Split the unseen data into a fine-tuning training set and a test set.
# X_new_train, X_new_test, y_new_train, y_new_test = train_test_split(
#     X_new, y_new, test_size=0.5, random_state=42, stratify=y_new
# )

# # Convert to PyTorch tensors.
# X_new_train = torch.tensor(X_new_train, dtype=torch.float)
# y_new_train = torch.tensor(y_new_train, dtype=torch.long)
# X_new_test  = torch.tensor(X_new_test, dtype=torch.float)
# y_new_test  = torch.tensor(y_new_test, dtype=torch.long)

# # ---------------------------
# # 2. Load the Pre-Trained Model from SetA
# # ---------------------------
# # Create a new model instance with the same architecture.
# class FraudMLP(nn.Module):
#     def __init__(self, input_dim, hidden_dim, output_dim):
#         super(FraudMLP, self).__init__()
#         self.model = nn.Sequential(
#             nn.Linear(input_dim, hidden_dim),
#             nn.ReLU(),
#             nn.BatchNorm1d(hidden_dim),
#             nn.Linear(hidden_dim, hidden_dim),
#             nn.ReLU(),
#             nn.BatchNorm1d(hidden_dim),
#             nn.Linear(hidden_dim, output_dim)
#         )
        
#     def forward(self, x):
#         return self.model(x)

# input_dim = X_new_train.shape[1]
# hidden_dim = 64
# output_dim = 2  # Two neurons for two classes

# model = FraudMLP(input_dim, hidden_dim, output_dim)

# loaded_model = FraudMLP(input_dim, hidden_dim, output_dim)
# loaded_model.load_state_dict(torch.load(best_model_path))

# # It's often useful to set the model in train mode during fine-tuning.
# loaded_model.train()

# # ---------------------------
# # 3. Fine-Tuning on Unseen Data (SetB)
# # ---------------------------
# # Use a lower learning rate for fine-tuning.
# finetune_optimizer = optim.Adam(loaded_model.parameters(), lr=1e-3)
# finetune_criterion = nn.CrossEntropyLoss()

# finetune_epochs = 300  # Adjust as needed

# print("\n--- Fine-Tuning on Unseen Data (SetB) ---")
# for epoch in range(finetune_epochs):
#     finetune_optimizer.zero_grad()
#     outputs = loaded_model(X_new_train)
#     loss = finetune_criterion(outputs, y_new_train)
#     loss.backward()
#     finetune_optimizer.step()
    
#     if (epoch + 1) % 5 == 0:
#         print(f"Fine-Tuning Epoch [{epoch + 1}/{finetune_epochs}], Loss: {loss.item():.4f}")


In [ ]:
# # ---------------------------
# # 4. Evaluate the Fine-Tuned Model on SetB Test Data
# # ---------------------------
# loaded_model.eval()
# with torch.no_grad():
#     test_outputs = loaded_model(X_new_test)
#     _, predicted = torch.max(test_outputs, 1)
#     y_pred = predicted.numpy()
#     y_true = y_new_test.numpy()
    
#     print("\nEvaluation on Fine-Tuned Unseen Data (SetB Test):")
#     print(classification_report(y_true, y_pred))
#     print(confusion_matrix(y_true, y_pred))
#     print("F1-score:", f1_score(y_true, y_pred))
#     print("Precision-score:", precision_score(y_true, y_pred))
#     print("Recall-score:", recall_score(y_true, y_pred))